In [1]:
import torch
from tqdm.notebook import tqdm

In [2]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, filename):
        self.data = []
        with open(filename, 'r') as f:
            for line in f:
                numbers = [float(num) for num in line.strip().split()]
                self.data.append((numbers[:4], numbers[4:]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs, targets = self.data[idx]
        return torch.tensor(inputs), torch.tensor(targets)

In [3]:
filename = "data/sph_100_10_20.txt"
full_dataset = CustomDataset(filename)

train_size = int(0.8 * len(full_dataset))
rest_size = len(full_dataset) - train_size
val_size = rest_size // 2
test_size = rest_size // 2
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size, test_size])

In [4]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32
train_shuffle = True
val_shuffle = False

dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=train_shuffle)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=val_shuffle)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
for inputs, targets in dataloader:
    assert inputs.shape[1] == 4 and targets.shape[1] == 2

In [6]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(4, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        return self.layers(x)

In [7]:
LR = 5e-5
NUM_EPOCHS = int(1e4)
MOMENTUM = 0.9
WEIGHT_DECAY = 0.999

In [8]:
model = MLP()
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=WEIGHT_DECAY)

In [9]:
from torch.utils.tensorboard import SummaryWriter

def train(model, dataloader, optimizer, loss_fn, epoch, writer, log_perc = 0.1):
    model.train()
    total_loss = 0
    total_diff = 0

    logs_steps = int(log_perc * len(dataloader))
    start_step = epoch * len(dataloader)

    for idx, (inputs, targets) in enumerate(dataloader):
        optimizer.zero_grad()

        outputs = model(inputs)

        loss = loss_fn(outputs, targets)
        diff = torch.abs(outputs - targets).mean()

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_diff += diff.item()
        
        if idx % logs_steps == 0:
            writer.add_scalar('Loss/Train', loss.item(), start_step + idx)
            writer.add_scalar('Absolute Difference/Train', diff.item(), start_step + idx)

    average_loss = total_loss / len(dataloader)
    average_diff = total_diff / len(dataloader)

    writer.add_scalar('Avg_Loss/Train', average_loss, epoch)
    writer.add_scalar('Avg_Absolute Difference/Train', average_diff, epoch)

    # print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Train Loss: {average_loss:.4f}, Train Diff: {average_diff:.15f}")

In [10]:
def validate(model, dataloader, loss_fn, epoch, writer):
    model.eval()
    total_loss = 0
    total_diff = 0

    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)

            loss = loss_fn(outputs, targets)
            diff = torch.abs(outputs - targets).mean()
            
            total_loss += loss.item()
            total_diff += diff.item()

    average_loss = total_loss / len(dataloader)
    average_diff = total_diff / len(dataloader)

    if writer is not None:
        writer.add_scalar('Avg_Loss/Val', average_loss, epoch)
        writer.add_scalar('Avg_Absolute Difference/Val', average_diff, epoch)

    if epoch is not None:
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Val Loss: {average_loss:.4f}, Val Diff: {average_diff:.15f}")
    else:
        print(f"Test Loss: {average_loss:.4f}, Test Diff: {average_diff:.15f}")

In [11]:
import datetime

writer = SummaryWriter(f"tb_logs/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}")

VALIDATION_STEPS = NUM_EPOCHS / 10

for idx, epoch in enumerate(tqdm(range(NUM_EPOCHS))):
    train(model, dataloader, optimizer, loss_fn, epoch, writer)

    if idx % VALIDATION_STEPS == 0:
        validate(model, val_dataloader, loss_fn, epoch, writer)

# After training, launch TensorBoard: `tensorboard --logdir=tb_logs`

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch [1/10000] Val Loss: 0.0383, Val Diff: 0.129011051906716
Epoch [1001/10000] Val Loss: 0.0381, Val Diff: 0.121592423302077
Epoch [2001/10000] Val Loss: 0.0381, Val Diff: 0.121573685801455


: 

In [ ]:
validate(model, test_dataloader, loss_fn, epoch, writer)

  0%|          | 0/84 [00:00<?, ?it/s]

Epoch [10/10] Val Loss: 0.0405, Val Diff: 0.121164027335388
